In [1]:
from __future__ import print_function
import numpy as np
import h5py
import pickle as pkl
import random
import torch
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
from torch.distributions import normal
import itertools
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import pandas as pd
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
import scipy.spatial as sp
import pyfftw
import Pk_library as PKL
import MAS_library as MASL

/share/apps/h5py/2.7.0rc2/intel/lib/python2.7/site-packages/h5py-2.7.0rc2-py2.7-linux-x86_64.egg/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
redshift_df = pd.DataFrame(pd.read_csv("/scratch/adc563/final_/redshift_df_5.csv"))

In [3]:
redshift_df.head()

,HI_mass,position_x,position_y,position_z,radius,total_mass,voxel_x,voxel_y,voxel_z,voxel_radius,mhi_per_part,r_0,alpha,density_profile
0,6.017072e+10,52.565430,70.709950,41.806717,0.342796,2.717073e+12,1435.0,1930.0,1141.0,9.360607,6.017072e+07,0.006607,3.0,1.465240e+09
1,5.771502e+10,27.313435,68.982700,74.505150,0.184113,2.579728e+12,745.0,1883.0,2034.0,5.027502,5.771502e+07,0.006607,3.0,8.921816e+09
2,5.644856e+10,52.644394,10.293198,34.506596,0.291465,2.509526e+12,1437.0,281.0,942.0,7.958927,5.644856e+07,0.006607,3.0,2.228695e+09
3,5.156705e+10,10.922141,29.594847,71.926700,0.266747,2.243074e+12,298.0,808.0,1964.0,7.283962,5.156705e+07,0.006607,3.0,2.650443e+09
4,4.471247e+10,72.529680,11.999981,18.106144,0.267865,1.880590e+12,1980.0,327.0,494.0,7.314492,4.471247e+07,0.006607,3.0,2.269709e+09


In [4]:
def get_particle_position_density(df, n_particles, epsilon=1e-3):
    num_halos = df.shape[0]
    n_particles = int(n_particles)
    center_x, center_y, center_z = df["position_x"], df["position_y"], df["position_z"]
    U = np.random.rand(num_halos, n_particles)
    V = np.random.rand(num_halos, n_particles)
    W = np.random.rand(num_halos, n_particles)
    print ("random numbers generated.")
    r = epsilon*(np.power(np.array(df["radius"]).reshape(-1,1)/epsilon, U)) # voxel r
    phi = np.multiply(np.ones((num_halos, 1)), 2*np.pi*V)
    theta = np.multiply(np.ones((num_halos, 1)), np.arccos(-1+2*W))
    print ("r, phi, theta generated.")
    x = np.array(center_x).reshape(-1,1) + np.multiply(np.multiply(r, np.sin(theta)), np.cos(phi))
    y = np.array(center_y).reshape(-1,1) + np.multiply(np.multiply(r, np.sin(theta)), np.sin(phi))
    z = np.array(center_z).reshape(-1,1) + np.multiply(r, np.cos(theta))
    print ("x, y, z generated. stacking started.")
    particles_in_halos = np.stack((x,y,z), axis=2)
    return particles_in_halos

In [5]:
def write_mhi_per_particle(df, n_particles):
    df["mhi_per_part"] = df["HI_mass"].apply(lambda x: x/n_particles)
    return df

def get_dens_list(df, n_particles):
    position_density = []
    for i in range(len(df)):
        density = df["mhi_per_part"].iloc[i]
        position_density.extend([density for x in range(n_particles)])
    return np.array(position_density)

In [7]:
particles_in_halos = get_particle_position_density(redshift_df, 200)
particles_in_halos = np.stack((particles_in_halos), axis=1)
particles_in_halos = particles_in_halos.reshape(4693294*200,3)

density_list = get_dens_list(redshift_df, 200)
print ("density list ready")

mask = (particles_in_halos>=0)&(particles_in_halos<=75)
print (mask.shape)
mask = mask.all(1)
print (mask.shape)
print ("masking.")

particles_in_halos = particles_in_halos[mask]
print (particles_in_halos.shape)
density_list = density_list[mask]
print (density_list.shape)
print ("saving files.")
# change
h5f0 = h5py.File('particles_in_halos_1.h5', 'w')
h5f0.create_dataset('data', data=particles_in_halos)
h5f0.close()

h5f1 = h5py.File('density_1.h5', 'w')
h5f1.create_dataset('data', data=density_list) 
h5f1.close()

random numbers generated.
r, phi, theta generated.
x, y, z generated. stacking started.
density list ready
(938658800, 3)
(938658800,)
masking.
(938544504, 3)
(938544504,)
saving files.


In [8]:
particles_in_halos[:5]

array([[52.56479178, 70.70140402, 41.80993439],
       [27.29242354, 69.02345436, 74.53940022],
       [52.7502848 , 10.17180977, 34.58581728],
       [10.90888475, 29.59061221, 71.90830348],
       [72.54234827, 11.95751671, 18.10588901]])

In [6]:
particles_in_halos = get_particle_position_density(redshift_df, 200)
particles_in_halos = np.stack((particles_in_halos), axis=1)
particles_in_halos = particles_in_halos.reshape(4693294*200,3)

density_list = get_dens_list(redshift_df, 200)
print ("density list ready")

mask = (particles_in_halos>=0)&(particles_in_halos<=75)
print (mask.shape)
mask = mask.all(1)
print (mask.shape)
print ("masking.")

particles_in_halos = particles_in_halos[mask]
print (particles_in_halos.shape)
density_list = density_list[mask]
print (density_list.shape)
print ("saving files.")
# change
h5f0 = h5py.File('particles_in_halos_2.h5', 'w')
h5f0.create_dataset('data', data=particles_in_halos)
h5f0.close()

h5f1 = h5py.File('density_2.h5', 'w')
h5f1.create_dataset('data', data=density_list)
h5f1.close()

random numbers generated.
r, phi, theta generated.
x, y, z generated. stacking started.
density list ready
(938658800, 3)
(938658800,)
masking.
(938544244, 3)
(938544244,)
saving files.


In [7]:
particles_in_halos[:5]

array([[52.56537948, 70.71177445, 41.80807988],
       [27.29269192, 68.94130118, 74.48928017],
       [52.64532968, 10.29012217, 34.50782292],
       [10.87745116, 29.59810839, 71.93384831],
       [72.51489586, 12.00457256, 18.12148813]])

In [6]:
particles_in_halos = get_particle_position_density(redshift_df, 200)
particles_in_halos = np.stack((particles_in_halos), axis=1)
particles_in_halos = particles_in_halos.reshape(4693294*200,3)

density_list = get_dens_list(redshift_df, 200)
print ("density list ready")

mask = (particles_in_halos>=0)&(particles_in_halos<=75)
print (mask.shape)
mask = mask.all(1)
print (mask.shape)
print ("masking.")

particles_in_halos = particles_in_halos[mask]
print (particles_in_halos.shape)
density_list = density_list[mask]
print (density_list.shape)
print ("saving files.")
# change
h5f0 = h5py.File('particles_in_halos_3.h5', 'w')
h5f0.create_dataset('data', data=particles_in_halos)
h5f0.close()

h5f1 = h5py.File('density_3.h5', 'w')
h5f1.create_dataset('data', data=density_list)
h5f1.close()

random numbers generated.
r, phi, theta generated.
x, y, z generated. stacking started.
density list ready
(938658800, 3)
(938658800,)
masking.
(938544845, 3)
(938544845,)
saving files.


In [7]:
particles_in_halos[:5]

array([[52.53692679, 70.73299461, 41.80167649],
       [27.30973922, 68.98333357, 74.50270586],
       [52.73538359, 10.30334167, 34.54525276],
       [10.75481916, 29.50884321, 71.99957015],
       [72.65190345, 11.96005057, 18.07773587]])

In [6]:
# RUN
particles_in_halos = get_particle_position_density(redshift_df, 200)
particles_in_halos = np.stack((particles_in_halos), axis=1)
particles_in_halos = particles_in_halos.reshape(4693294*200,3)

density_list = get_dens_list(redshift_df, 200)
print ("density list ready")

mask = (particles_in_halos>=0)&(particles_in_halos<=75)
print (mask.shape)
mask = mask.all(1)
print (mask.shape)
print ("masking.")

particles_in_halos = particles_in_halos[mask]
print (particles_in_halos.shape)
density_list = density_list[mask]
print (density_list.shape)
print ("saving files.")
# change
h5f0 = h5py.File('particles_in_halos_4.h5', 'w')
h5f0.create_dataset('data', data=particles_in_halos)
h5f0.close()

h5f1 = h5py.File('density_4.h5', 'w')
h5f1.create_dataset('data', data=density_list)
h5f1.close()

random numbers generated.
r, phi, theta generated.
x, y, z generated. stacking started.
density list ready
(938658800, 3)
(938658800,)
masking.
(938544235, 3)
(938544235,)
saving files.


In [7]:
particles_in_halos[:5]

array([[52.5633382 , 70.70964935, 41.8076254 ],
       [27.31835465, 68.98228205, 74.51232922],
       [52.65090529, 10.28942945, 34.52664456],
       [10.91542927, 29.6324498 , 71.93854608],
       [72.54258841, 11.99061174, 18.10698325]])

In [6]:
particles_in_halos = get_particle_position_density(redshift_df, 200)
particles_in_halos = np.stack((particles_in_halos), axis=1)
particles_in_halos = particles_in_halos.reshape(4693294*200,3)

density_list = get_dens_list(redshift_df, 200)
print ("density list ready")

mask = (particles_in_halos>=0)&(particles_in_halos<=75)
print (mask.shape)
mask = mask.all(1)
print (mask.shape)
print ("masking.")

particles_in_halos = particles_in_halos[mask]
print (particles_in_halos.shape)
density_list = density_list[mask]
print (density_list.shape)
print ("saving files.")
# change
h5f0 = h5py.File('particles_in_halos_5.h5', 'w')
h5f0.create_dataset('data', data=particles_in_halos)
h5f0.close()

h5f1 = h5py.File('density_5.h5', 'w')
h5f1.create_dataset('data', data=density_list)
h5f1.close()

random numbers generated.
r, phi, theta generated.
x, y, z generated. stacking started.
density list ready
(938658800, 3)
(938658800,)
masking.
(938544938, 3)
(938544938,)
saving files.


In [7]:
particles_in_halos[:5]

array([[52.56523727, 70.70998015, 41.80542553],
       [27.28391896, 68.97296089, 74.51881229],
       [52.61992896, 10.25184997, 34.51566094],
       [10.95254577, 29.5644444 , 71.92763051],
       [72.56178779, 12.05531135, 18.08314273]])

In [ ]:
# write delta to h5py

In [2]:
# pos1 = h5py.File('particles_in_halos_1.h5', 'r')["data"][:]
# print ("pos 1/5 read.")
# pos2 = h5py.File('particles_in_halos_2.h5', 'r')["data"][:]
# print ("pos 2/5 read.")
# pos3 = h5py.File('particles_in_halos_3.h5', 'r')["data"][:]
# print ("pos 3/5 read.")
# pos4 = h5py.File('particles_in_halos_4.h5', 'r')["data"][:]
# print ("pos 4/5 read.")
pos5 = h5py.File('particles_in_halos_5.h5', 'r')["data"][:]
print ("pos 5/5 read.")

# dens1 = h5py.File('density_1.h5', 'r')["data"][:]
# print ("dens 1/5 read.")
# dens2 = h5py.File('density_2.h5', 'r')["data"][:]
# print ("dens 2/5 read.")
# dens3 = h5py.File('density_3.h5', 'r')["data"][:]
# print ("dens 3/5 read.")
# dens4 = h5py.File('density_4.h5', 'r')["data"][:]
# print ("dens 4/5 read.")
dens5 = h5py.File('density_5.h5', 'r')["data"][:]
print ("dens 5/5 read.")

pos 5/5 read.
dens 5/5 read.


In [3]:
BoxSize = 75.0
# pos = np.vstack((pos1,pos2,pos3,pos4,pos5))
# mhi = np.vstack((dens1,dens2,dens3,dens4,dens5))
delta = np.zeros((2048,2048,2048)).astype(np.float32)
# print (type(delta))
MASL.MA(pos1.astype(np.float32), delta, BoxSize, MAS="CIC", W=dens1.astype(np.float32))
print ("writing delta file.")
h5f1 = h5py.File('delta_mas_1.h5', 'w')
h5f1.create_dataset('data', data=delta)
h5f1.close()

writing delta file.


In [6]:
np.sum(delta[:64,:64,:64])

252587060.0

In [5]:
np.sum(delta[2000:,2000:,2000:])

0.0

In [3]:
# RUN
BoxSize = 75.0
# pos = np.vstack((pos1,pos2,pos3,pos4,pos5))
# mhi = np.vstack((dens1,dens2,dens3,dens4,dens5))
delta = h5py.File('delta_mas_1.h5', 'r')["data"][:]
MASL.MA(pos2.astype(np.float32), delta, BoxSize, MAS="CIC", W=dens2.astype(np.float32))
print ("writing delta file.")
h5f1 = h5py.File('delta_mas_2.h5', 'w')
h5f1.create_dataset('data', data=delta)
h5f1.close()

writing delta file.


In [4]:
np.sum(delta[:64,:64,:64])

505331100.0

In [5]:
np.sum(delta[2000:,2000:,2000:])

0.0

In [3]:
# RUN
BoxSize = 75.0
print ("start.")
# pos = np.vstack((pos1,pos2,pos3,pos4,pos5))
# mhi = np.vstack((dens1,dens2,dens3,dens4,dens5))
delta = h5py.File('delta_mas_2.h5', 'r')["data"][:]
print (type(delta))
MASL.MA(pos3.astype(np.float32), delta, BoxSize, MAS="CIC", W=dens3.astype(np.float32))
print ("writing delta file.")
h5f1 = h5py.File('delta_mas_3.h5', 'w')
h5f1.create_dataset('data', data=delta)
h5f1.close()

start.
<type 'numpy.ndarray'>
writing delta file.


In [7]:
np.sum(delta[:64,:64,:64])

758410800.0

In [8]:
np.sum(delta[2000:,2000:,2000:])

0.0

In [3]:
BoxSize = 75.0
# pos = np.vstack((pos1,pos2,pos3,pos4,pos5))
# mhi = np.vstack((dens1,dens2,dens3,dens4,dens5))
delta = h5py.File('delta_mas_3.h5', 'r')["data"][:]
print (type(delta))
MASL.MA(pos4.astype(np.float32), delta, BoxSize, MAS="CIC", W=dens4.astype(np.float32))
print ("writing delta file.")
h5f1 = h5py.File('delta_mas_4.h5', 'w')
h5f1.create_dataset('data', data=delta)
h5f1.close()

<type 'numpy.ndarray'>
writing delta file.


In [5]:
np.sum(delta[:64,:64,:64])

1011042100.0

In [4]:
np.sum(delta[2000:,2000:,2000:])

0.0

In [3]:
# RUN NEXT KERNEL
BoxSize = 75.0
# pos = np.vstack((pos1,pos2,pos3,pos4,pos5))
# mhi = np.vstack((dens1,dens2,dens3,dens4,dens5))
delta = h5py.File('delta_mas_4.h5', 'r')["data"][:]
print (type(delta))
MASL.MA(pos5.astype(np.float32), delta, BoxSize, MAS="CIC", W=dens5.astype(np.float32))
print ("writing delta file.")
h5f1 = h5py.File('delta_mas_5.h5', 'w')
h5f1.create_dataset('data', data=delta)
h5f1.close()

<type 'numpy.ndarray'>
writing delta file.


In [4]:
np.sum(delta[:64,:64,:64])

1264489900.0

In [5]:
np.sum(delta[2000:,2000:,2000:])

0.0

In [6]:
import itertools
path = "new_hod_grid"
edge_r = list(np.arange(0, 2048, 64))
lower_corners = list(itertools.product(edge_r, edge_r, edge_r))
for corner in lower_corners:
    with open("{}/lower_corner_{}_{}_{}.pkl".format(path, corner[0], corner[1], corner[2]),"wb") as f:
        pkl.dump(delta[corner[0]:corner[0]+64,corner[1]:corner[1]+64,corner[2]:corner[2]+64], 
                 f, protocol = 2)